# Preparación del entorno: instalaciones y librerías

In [1]:
#Instalaciones
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
#Librerías
import os
import pandas as pd
import glob
import shutil
from google.colab import drive
import torch.nn.functional as F
from torch.optim import Adam

#Semilla
SEED = 42

#Librerías donde fijar semilla para garantizar reproducibilidad
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)
import torch
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback, set_seed
set_seed(SEED)
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, f1_score
from sklearn.model_selection import ParameterGrid

# Dataset

In [3]:
#Carga del dataset de España
dataset_es = load_dataset("alttal/balanced_SA_opiniones_indumentaria_ES_Espanya_splits")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3006 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/625 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/313 [00:00<?, ? examples/s]

In [4]:
dataset_es

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 3006
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 625
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 313
    })
})

# Métricas de evaluación

In [5]:
#Definimos las métricas para evaluar el modelo
#Este conjunto de métricas depende de la tarea.
#Para clasificación de textos se suelen utilizar accuracy, precision, recall y F1-score.

def compute_metrics(pred):
  y_true = pred.label_ids              # son las etiquetas reales
  y_pred = pred.predictions.argmax(-1) # son las predicciones
  acc = accuracy_score(y_true, y_pred)
  precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

  #métricas por clase
  precision_per_class, recall_per_class, f1_per_class, _ = precision_recall_fscore_support(y_true, y_pred, average=None)

  #conversiones necesarias para evitar errores de registro en Tensorboard
  precision_per_class = list(precision_per_class)
  recall_per_class = list(recall_per_class)
  f1_per_class = list(f1_per_class)

  return {
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall,
      'precision_per_class': precision_per_class,
      'recall_per_class': recall_per_class,
      'f1_per_class': f1_per_class
  }

# Documentación del progreso

In [6]:
#Montar unidad de disco
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#Ruta raíz para guardar cada experimento
path = '/content/drive/My Drive/TFM/Experimentos balanced'

#Cambiar directorio actual de trabajo
os.chdir(path)

#Verificar directorio actual
print("Directorio actual:", os.getcwd())

Directorio actual: /content/drive/My Drive/TFM/Experimentos balanced


In [8]:
#Definición de una función para eliminar logs antiguos (para evitar falta de almacenamiento)
def clean_old_logs(logging_dir, keep_last_n=1):
    log_files = sorted(glob.glob(os.path.join(logging_dir, '*')), key=os.path.getmtime)
    for log_file in log_files[:-keep_last_n]:
        os.remove(log_file)

In [9]:
#Ruta al directorio de logs
log_dir = path

#Eliminar archivos antiguos de TensorBoard (para evitar acumulación de demasiados logs)
for file in os.listdir(log_dir):
    if file.startswith('events.out.tfevents'):
        os.remove(os.path.join(log_dir, file))

# Modelo 3

##Carga del modelo 3 y adaptación de los datos

In [11]:
#Carga del modelo pre-entrenado
model3 = AutoModelForSequenceClassification.from_pretrained("edumunozsala/bertin_base_sentiment_analysis_es", num_labels=3, ignore_mismatched_sizes=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at edumunozsala/bertin_base_sentiment_analysis_es and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
#A modo de información, vemos la configuración del modelo
model3.config

RobertaConfig {
  "_name_or_path": "edumunozsala/bertin_base_sentiment_analysis_es",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "transformers_version": "4.42.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [13]:
#¿Cómo son las etiquetas del modelo? Importante porque deben ser iguales a las del dataset
model3.config.id2label

{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2'}

In [14]:
#Modificar etiquetas
model3.config.id2label = {0: 'NEG', 1: 'NEU', 2: 'POS'}
model3.config.label2id = {v: k for k, v in model3.config.id2label.items()}

In [15]:
#¿Cómo son las etiquetas del modelo luego del cambio?
model3.config.id2label

{0: 'NEG', 1: 'NEU', 2: 'POS'}

##Tokenización del dataset

In [16]:
#Carga del tokenizador específico
tokenizer3 = AutoTokenizer.from_pretrained("edumunozsala/bertin_base_sentiment_analysis_es")

tokenizer_config.json:   0%|          | 0.00/332 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/855k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/514k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [17]:
#Se define tokenizador del primer modelo
def tokenize_function3(example):
    return tokenizer3(example["text"], padding='longest', truncation=True)

In [18]:
#Tokenización del dataset
encoded_dataset_es_1 = dataset_es.map(tokenize_function3, batched=True)
#Luego de tokenizar se puede eliminar la columna "text" porque el modelo no la necesita
encoded_dataset_es_1 = encoded_dataset_es_1.remove_columns(["text"])
encoded_dataset_es_1

Map:   0%|          | 0/3006 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/625 [00:00<?, ? examples/s]

Map:   0%|          | 0/313 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 3006
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 625
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 313
    })
})

In [19]:
#Ejemplo de la tokenización en cada split
print(encoded_dataset_es_1['train'][0])
print(encoded_dataset_es_1['validation'][0])
print(encoded_dataset_es_1['test'][0])

{'label': 0, 'input_ids': [1895, 378, 13855, 353, 2994, 293, 315, 642, 10027, 378, 5520, 11755, 359, 5520, 34545, 802, 283, 15173, 356, 263, 714, 610, 4880, 28822, 562, 426, 359, 316, 2611, 389, 2749, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,

## Experimento 3.1.

In [20]:
#Documentación
#Definición de directorios para el primer experimento
output_dir = "./outputs/model3_lit"
shutil.make_archive("./outputs/model3_lit", 'zip', output_dir)
logging_dir = "./logs/model3_lit"
shutil.make_archive("./logs/model3_lit", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

Entrenamiento con hiperparámetros recomendados

In [21]:
#Definición de EarlyStopping
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,    # Número de épocas para esperar sin mejora
    early_stopping_threshold=0.0  # Umbral para la mejora mínima
)

In [22]:
#Definición de hiperparámetros
training_args_model3_lit = TrainingArguments(
    output_dir=output_dir,           # Directorio de salida
    logging_dir=logging_dir,         # Directorio para los logs
    eval_strategy="steps",           # Estrategia de evaluación
    save_strategy="steps",           # Estrategia de guardado (debe coincidir con la de evaluación)
    save_steps=1000,                 # Número de pasos entre cada guardado
    logging_strategy="steps",        # Estrategia de registro
    logging_steps=1000,              # Número de pasos entre cada registro
    num_train_epochs=3,              # Número de épocas de entrenamiento
    per_device_train_batch_size=16,  # Tamaño del batch de entrenamiento (podría ser también 32)
    per_device_eval_batch_size=16,   # Tamaño del batch de evaluación (podría ser también 32)
    learning_rate=3e-5,              # Tasa de aprendizaje
    weight_decay=0.01,               # Decadencia de peso (podría ser también 0.1)
    warmup_ratio=0.1,                # Proporción de épocas de calentamiento
    save_total_limit=1,              # Número máximo de checkpoints a guardar (para evitar problemas de almacenamiento)
    seed=SEED,                         # Semilla para garantizar reproducibilidad
    fp16=True,                       # Uso de fp16 para acelerar el entrenamiento
    load_best_model_at_end=True,     # Cargar el mejor modelo al final del entrenamiento
    metric_for_best_model="f1",      # Métrica objetivo, a optimizar para encontrar el mejor modelo
)

In [23]:
#Definición de optimizador
optimizer = Adam(model3.parameters(), lr=3e-5, weight_decay=0.01)

In [24]:
#Definición de un objeto de la clase Trainer para el primer experimento con el modelo 1
trainer_model3_lit = Trainer(
    model = model3,
    args = training_args_model3_lit,
    train_dataset = encoded_dataset_es_1['train'],
    eval_dataset = encoded_dataset_es_1['validation'],
    compute_metrics=compute_metrics,
    tokenizer = tokenizer3,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer3),
    optimizers = (optimizer, None),
    callbacks = [early_stopping])

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Entrenamiento

In [25]:
trainer_model3_lit.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=564, training_loss=0.6564207144662844, metrics={'train_runtime': 43.3983, 'train_samples_per_second': 207.796, 'train_steps_per_second': 12.996, 'total_flos': 1334396142596160.0, 'train_loss': 0.6564207144662844, 'epoch': 3.0})

Validación

In [26]:
trainer_model3_lit.evaluate()

Trainer is attempting to log a value of "[0.9439252336448598, 0.47368421052631576, 0.9538461538461539]" of type <class 'list'> for key "eval/precision_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.808, 0.8, 0.8671328671328671]" of type <class 'list'> for key "eval/recall_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8706896551724138, 0.5950413223140496, 0.9084249084249084]" of type <class 'list'> for key "eval/f1_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.4147877097129822,
 'eval_accuracy': 0.8338658146964856,
 'eval_f1': 0.7913852953037906,
 'eval_precision': 0.7904851993391099,
 'eval_recall': 0.8250442890442891,
 'eval_precision_per_class': [0.9439252336448598,
  0.47368421052631576,
  0.9538461538461539],
 'eval_recall_per_class': [0.808, 0.8, 0.8671328671328671],
 'eval_f1_per_class': [0.8706896551724138,
  0.5950413223140496,
  0.9084249084249084],
 'eval_runtime': 0.5399,
 'eval_samples_per_second': 579.751,
 'eval_steps_per_second': 37.045,
 'epoch': 3.0}

Reporte de clasificación (sobre conjunto de validación)

In [27]:
#Obtención de predicciones del conjunto de validación
predictions = trainer_model3_lit.predict(encoded_dataset_es_1['validation'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.94      0.81      0.87       125
           1       0.47      0.80      0.60        45
           2       0.95      0.87      0.91       143

    accuracy                           0.83       313
   macro avg       0.79      0.83      0.79       313
weighted avg       0.88      0.83      0.85       313


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.94   0.81     0.87
    1      0.47   0.80     0.60
    2      0.95   0.87     0.91


In [28]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

Test

Reporte de clasificación (sobre conjunto de test)

In [29]:
#Obtención de predicciones del conjunto de test
predictions = trainer_model3_lit.predict(encoded_dataset_es_1['test'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.91      0.80      0.85       248
           1       0.45      0.73      0.55        90
           2       0.92      0.83      0.87       287

    accuracy                           0.80       625
   macro avg       0.76      0.79      0.76       625
weighted avg       0.85      0.80      0.82       625


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.91   0.80     0.85
    1      0.45   0.73     0.55
    2      0.92   0.83     0.87


In [30]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

## Grid Search

Búsqueda de hiperparámetros optimizados con la técnica Grid Search:

In [31]:
#Definición del espacio de búsqueda
param_grid = {
    'num_train_epochs': [3, 5],
    'per_device_train_batch_size': [16, 32],
    'per_device_eval_batch_size': [16, 32],
    'learning_rate': [1e-5, 2e-5, 3e-5, 5e-5, 1e-4],
    'weight_decay': [0.01, 0.1]
}

grid = ParameterGrid(param_grid)

In [32]:
#Función para calcular el f1-score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        'f1': f1_score(labels, predictions, average='macro')
    }

In [33]:
#Inicialización de variables para guardar los mejores resultados
best_score = 0
best_params = None

In [34]:
#Documentación
#Definición de directorios para el primer experimento
output_dir = "./outputs/model3_search"
shutil.make_archive("./outputs/model3_search", 'zip', output_dir)
logging_dir = "./logs/model3_search"
shutil.make_archive("./logs/model3_search", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

In [35]:
#Bucle para probar los diferentes parámetros
for params in grid:
    print(f"Entrenando con parámetros: {params}")

    #Definición de hiperparámetros
    training_args_model3_search = TrainingArguments(
        output_dir=output_dir,                                                  # Directorio de salida
        logging_dir=logging_dir,                                                # Directorio para los logs
        eval_strategy="steps",                                                  # Estrategia de evaluación
        save_strategy="steps",                                                  # Estrategia de guardado (debe coincidir con la de evaluación)
        save_steps=1000,                                                        # Número de pasos entre cada guardado
        logging_strategy="steps",                                               # Estrategia de registro
        logging_steps=1000,                                                     # Número de pasos entre cada registro
        num_train_epochs=params['num_train_epochs'],                            # Número de épocas de entrenamiento
        per_device_train_batch_size=params['per_device_train_batch_size'],      # Tamaño del batch de entrenamiento
        per_device_eval_batch_size=params['per_device_eval_batch_size'],        # Tamaño del batch de evaluación
        learning_rate=params['learning_rate'],                                  # Tasa de aprendizaje
        weight_decay=params['weight_decay'],                                    # Decadencia de peso
        save_total_limit=1,                                                     # Número máximo de checkpoints a guardar (para evitar problemas de almacenamiento)
        seed=SEED,                                                              # Semilla para garantizar reproducibilidad
        fp16=True,                                                              # Uso de fp16 para acelerar el entrenamiento
        load_best_model_at_end=True,                                            # Cargar el mejor modelo al final del entrenamiento
        metric_for_best_model='f1',                                             # Métrica objetivo, a optimizar para encontrar el mejor modelo
    )

    #Definición de un objeto de la clase Trainer para la búsqueda con el modelo 3
    trainer_model3_search = Trainer(
        model=model3,
        args=training_args_model3_search,
        train_dataset=encoded_dataset_es_1['train'],
        eval_dataset=encoded_dataset_es_1['validation'],
        tokenizer=tokenizer3,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer3),
        compute_metrics=compute_metrics,
        optimizers = (optimizer, None),
        callbacks = [early_stopping]
    )

    #Entrenamiento
    trainer_model3_search.train()

    #Limpiar logs antiguos después del entrenamiento en cada iteración
    clean_old_logs(logging_dir, keep_last_n=1)

    #Validación
    eval_result = trainer_model3_search.evaluate()

    #Guardamos métrica objetivo
    score = eval_result.get('eval_f1', 0)

    #Siempre que la métrica objetivo sea mejor que el score guardado hasta el momento, se lo actualiza
    if score > best_score:
        best_score = score
        best_params = params

#Vemos los resultados del Grid Search
print(f"Mejores parámetros: {best_params}")
print(f"Mejor puntuación F1: {best_score}")

Entrenando con parámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'weight_decay': 0.01}


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss,Validation Loss


Entrenando con parámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'weight_decay': 0.1}


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


...

Step,Training Loss,Validation Loss


Entrenando con parámetros: {'learning_rate': 0.0001, 'num_train_epochs': 5, 'per_device_eval_batch_size': 32, 'per_device_train_batch_size': 32, 'weight_decay': 0.1}


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss,Validation Loss


Mejores parámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'weight_decay': 0.1}
Mejor puntuación F1: 0.7807368491951235


## Experimento 3.2.

In [36]:
#Documentación
#Definición de directorios para el segundo experimento
output_dir = "./outputs/model3_opt"
shutil.make_archive("./outputs/model3_opt", 'zip', output_dir)
logging_dir = "./logs/mode31_opt"
shutil.make_archive("./logs/model3_opt", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

Entrenamiento con hiperparámetros optimizados

In [37]:
#Definición de EarlyStopping
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,    # Número de épocas para esperar sin mejora
    early_stopping_threshold=0.0  # Umbral para la mejora mínima
)

In [38]:
#Definición de hiperparámetros (basados en Grid Search)
training_args_model3_opt = TrainingArguments(
    output_dir=output_dir,                                # Directorio de salida
    logging_dir=logging_dir,                              # Directorio para los logs
    eval_strategy="steps",                                # Estrategia de evaluación
    save_strategy="steps",                                # Estrategia de guardado (debe coincidir con la de evaluación)
    save_steps=1000,                                      # Número de pasos entre cada guardado
    logging_strategy="steps",                             # Estrategia de registro
    logging_steps=1000,                                   # Número de pasos entre cada registro
    num_train_epochs=3,                                   # Número de épocas de entrenamiento
    per_device_train_batch_size=16,                       # Tamaño del batch de entrenamiento
    per_device_eval_batch_size=16,                        # Tamaño del batch de evaluación
    learning_rate=1e-05,                                  # Tasa de aprendizaje
    weight_decay=0.1,                                     # Decadencia de peso
    save_total_limit=1,                                   # Número máximo de checkpoints a guardar (para evitar problemas de almacenamiento)
    seed=SEED,                                            # Semilla para garantizar reproducibilidad
    fp16=True,                                            # Uso de fp16 para acelerar el entrenamiento
    load_best_model_at_end=True,                          # Cargar el mejor modelo al final del entrenamiento
    metric_for_best_model="f1",                           # Métrica objetivo, a optimizar para encontrar el mejor modelo
)

In [39]:
#Definición de optimizador
optimizer = Adam(model3.parameters(), lr=params['learning_rate'], weight_decay=params['weight_decay'])

In [40]:
#Definición de un objeto de la clase Trainer para el segundo experimento con el modelo 1
trainer_model3_opt = Trainer(
    model = model3,
    args = training_args_model3_opt,
    train_dataset = encoded_dataset_es_1['train'],
    eval_dataset = encoded_dataset_es_1['validation'],
    compute_metrics=compute_metrics,
    tokenizer = tokenizer3,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer3),
    optimizers = (optimizer, None),
    callbacks = [early_stopping])

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Entrenamiento

In [41]:
trainer_model3_opt.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=564, training_loss=1.0832592037552637, metrics={'train_runtime': 46.9389, 'train_samples_per_second': 192.122, 'train_steps_per_second': 12.016, 'total_flos': 1334396142596160.0, 'train_loss': 1.0832592037552637, 'epoch': 3.0})

Validación

In [42]:
trainer_model3_opt.evaluate()

{'eval_loss': 1.0816459655761719,
 'eval_f1': 0.38653813630982586,
 'eval_runtime': 0.4999,
 'eval_samples_per_second': 626.182,
 'eval_steps_per_second': 40.012,
 'epoch': 3.0}

Reporte de clasificación (sobre conjunto de validación)

In [43]:
#Obtención de predicciones del conjunto de validación
predictions = trainer_model3_opt.predict(encoded_dataset_es_1['validation'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.62      0.45      0.52       125
           1       0.00      0.00      0.00        45
           2       0.53      0.82      0.64       143

    accuracy                           0.55       313
   macro avg       0.38      0.42      0.39       313
weighted avg       0.49      0.55      0.50       313


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.62   0.45     0.52
    1      0.00   0.00     0.00
    2      0.53   0.82     0.64


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [44]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

Test

Reporte de clasificación (sobre conjunto de test)

In [45]:
#Obtención de predicciones del conjunto de test
predictions = trainer_model3_opt.predict(encoded_dataset_es_1['test'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.52      0.37      0.43       248
           1       0.00      0.00      0.00        90
           2       0.51      0.81      0.63       287

    accuracy                           0.52       625
   macro avg       0.35      0.39      0.35       625
weighted avg       0.44      0.52      0.46       625


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.52   0.37     0.43
    1      0.00   0.00     0.00
    2      0.51   0.81     0.63


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [46]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

## Experimento extra 3.3.

In [47]:
#Documentación
#Definición de directorios para el tercer experimento
output_dir = "./outputs/model3_est"
shutil.make_archive("./outputs/model3_est", 'zip', output_dir)
logging_dir = "./logs/model3_est"
shutil.make_archive("./logs/model3_est", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

Entrenamiento con hiperparámetros originales

In [48]:
#Definición de hiperparámetros estándar
training_args_model3_estandar = TrainingArguments(
    output_dir=output_dir,
    logging_dir=logging_dir,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=1000,
    logging_strategy="steps",
    logging_steps=1000,
    seed=42,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

In [49]:
#Definición de un objeto de la clase Trainer para el tercer experimento con el modelo 1
trainer_model3_estandar = Trainer(
    model = model3,
    args = training_args_model3_estandar,
    train_dataset = encoded_dataset_es_1['train'],
    eval_dataset = encoded_dataset_es_1['validation'],
    compute_metrics=compute_metrics,
    tokenizer = tokenizer3,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer3)
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Entrenamiento

In [50]:
trainer_model3_estandar.train()

Step,Training Loss,Validation Loss,F1
1000,1.052800,1.055564,0.396486


TrainOutput(global_step=1128, training_loss=1.049832080272918, metrics={'train_runtime': 76.6199, 'train_samples_per_second': 117.698, 'train_steps_per_second': 14.722, 'total_flos': 1328245866456768.0, 'train_loss': 1.049832080272918, 'epoch': 3.0})

Validación

In [51]:
trainer_model3_estandar.evaluate()

{'eval_loss': 1.0555641651153564,
 'eval_f1': 0.3964859738444644,
 'eval_runtime': 0.9237,
 'eval_samples_per_second': 338.838,
 'eval_steps_per_second': 43.302,
 'epoch': 3.0}

Reporte de clasificación (sobre conjunto de validación)

In [52]:
#Obtención de predicciones del conjunto de validación
predictions = trainer_model3_estandar.predict(encoded_dataset_es_1['validation'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.51      0.78      0.62       125
           1       0.00      0.00      0.00        45
           2       0.62      0.53      0.57       143

    accuracy                           0.55       313
   macro avg       0.38      0.44      0.40       313
weighted avg       0.49      0.55      0.51       313


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.51   0.78     0.62
    1      0.00   0.00     0.00
    2      0.62   0.53     0.57


In [53]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

Test

Reporte de clasificación (sobre conjunto de test)

In [54]:
#Obtención de predicciones del conjunto de test
predictions = trainer_model3_estandar.predict(encoded_dataset_es_1['test'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.48      0.71      0.57       248
           1       0.25      0.01      0.02        90
           2       0.58      0.52      0.55       287

    accuracy                           0.52       625
   macro avg       0.44      0.41      0.38       625
weighted avg       0.50      0.52      0.48       625


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.48   0.71     0.57
    1      0.25   0.01     0.02
    2      0.58   0.52     0.55


In [55]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

## Experimento final: mejor modelo, nuevo dataset

In [56]:
#Elección del mejor modelo
#Modelo final -> Modelo con hiperparámetros recomendados
#Carga del nuevo dataset: comentarios de Argentina
dataset_ar = load_dataset("alttal/SA_opiniones_indumentaria_ES_Argentina_splits")

Generating train split:   0%|          | 0/2187 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/625 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/313 [00:00<?, ? examples/s]

In [57]:
#Unimos los splits: sólo necesitaremos uno para el test
from datasets import concatenate_datasets
dataset_ar = concatenate_datasets([dataset_ar['train'], dataset_ar['test'], dataset_ar['validation']])

#Ejemplo
print(dataset_ar)

Dataset({
    features: ['id', 'label', 'text', '__index_level_0__'],
    num_rows: 3125
})


In [58]:
#Se eliminan columnas innecesarias
dataset_ar = dataset_ar.remove_columns(["id","__index_level_0__"])
dataset_ar

Dataset({
    features: ['label', 'text'],
    num_rows: 3125
})

In [59]:
#Las etiquetas de los datasets son strings. Se las cambia a valores numéricos
def label_to_int(label):
    if label == "POS":
        return 2
    elif label == "NEG":
        return 0
    else:
        return 1

dataset_ar_1 = dataset_ar.map(lambda x: {"label": label_to_int(x["label"])})

Map:   0%|          | 0/3125 [00:00<?, ? examples/s]

In [60]:
#Tokenización del dataset
encoded_dataset_ar_1 = dataset_ar_1.map(tokenize_function3, batched=True)
#Luego de tokenizar se puede eliminar la columna "text" porque el modelo no la necesita
encoded_dataset_ar_1 = encoded_dataset_ar_1.remove_columns(["text"])
encoded_dataset_ar_1

Map:   0%|          | 0/3125 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 3125
})

Reporte de clasificación sobre el conjunto de datos de Argentina

In [61]:
#Obtención de predicciones del conjunto de test
predictions = trainer_model3_lit.predict(encoded_dataset_es_1['test'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.48      0.71      0.57       248
           1       0.25      0.01      0.02        90
           2       0.58      0.52      0.55       287

    accuracy                           0.52       625
   macro avg       0.44      0.41      0.38       625
weighted avg       0.50      0.52      0.48       625


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.48   0.71     0.57
    1      0.25   0.01     0.02
    2      0.58   0.52     0.55
